In [11]:
import numpy as np
import threading
from threading import Thread
import pandas as pd
import re
import nltk
import spacy
import string
import queue
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from sklearn.decomposition import TruncatedSVD
from io import StringIO
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import time
from collections import Counter
from subprocess import call

In [2]:
df = pd.read_csv('1 Billion Citation Dataset, v1 (112).csv') 
# df = pd.read_csv('file_3.csv') 
df = pd.DataFrame(df)


In [3]:
df1 = df[:200000]
df2 = df[200000:400000]
df3 = df[400000:600000]
df4 = df[600000:800000]
df5 = df[800000:1000000]
df6 = df[1000000:1200000]
df7 = df[1200000:1400000]
df8 = df[1400000:1600000]
df9 = df[1600000:1800000]
df0 = df[1800000:2000000]


In [4]:
def remove_urls(text):
  url_pattern = re.compile(r"https?://\S+|www\.\S+")
  return url_pattern.sub(r"", text)

def remove_html(text):
  html_pattern = re.compile("<.*?>")
  return html_pattern.sub(r"", text)

PUNCT = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","" , PUNCT))

PUNCT = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","" , PUNCT))

nltk.download('stopwords')
",".join(stopwords.words("english"))
STOPW = set(stopwords.words("english"),)
def remove_stopwords(text):
  return" ".join([word for word in str(text).split() if word not in STOPW])        

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admins\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def preprocess(df):
    # Convert_lowerCase(df, "citationStringAnnotated")
    # df["Preprocess"] = df["citationStringAnnotated"].apply(lambda text: stem_words(text))
    # df["Preprocess"] = df["Preprocess"].apply(lambda text: lemmatize_words(text))
    df_preprocess = df.loc[:,'citationStringAnnotated']
    df_preprocess = df_preprocess.astype(str)
    df_preprocess = df_preprocess.str.lower()
    df_preprocess = df_preprocess.apply(lambda text: remove_urls(text))
    df_preprocess = df_preprocess.apply(lambda text: remove_html(text))
    df_preprocess= df_preprocess.apply(lambda text: remove_punctuation(text))
    df_preprocess = df_preprocess.apply(lambda text: remove_stopwords(text))
    df_preprocess = df_preprocess.str.replace('\d+', '',regex=True)

    return df_preprocess

In [6]:
class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
    def run(self):
        print(type(self._target))
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [7]:
# try:
start_time = time.time()
dfa = []
thread1 = ThreadWithReturnValue(target = preprocess, args=(df1,))
thread2 = ThreadWithReturnValue(target = preprocess, args=(df2,))
thread3 = ThreadWithReturnValue(target = preprocess, args=(df3,))
thread4 = ThreadWithReturnValue(target = preprocess, args=(df4,))
thread5 = ThreadWithReturnValue(target = preprocess, args=(df5,))
thread6 = ThreadWithReturnValue(target = preprocess, args=(df6,))
thread7 = ThreadWithReturnValue(target = preprocess, args=(df7,))
thread8 = ThreadWithReturnValue(target = preprocess, args=(df8,))
thread9 = ThreadWithReturnValue(target = preprocess, args=(df9,))
thread0 = ThreadWithReturnValue(target = preprocess, args=(df0,))

thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread5.start()
thread6.start()
thread7.start()
thread8.start()
thread9.start()
thread0.start()

df1 = thread1.join()
df2 = thread2.join()
df3 = thread3.join()
df4 = thread4.join()
df5 = thread5.join()
df6 = thread6.join()
df7 = thread7.join()
df8 = thread8.join()
df9 = thread9.join()
df0 = thread0.join()

# dfa = pd.concat((df1,df2,df3,df4,df5,df6,df7,df8,df9,df0), axis = 0)
print("--- %s seconds ---" % (time.time() - start_time))
# print(dfb)
# except:
#     print("error")

<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
--- 73.51923108100891 seconds ---


In [9]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df0]
  
result = pd.concat(frames)
display(result)

0          yorizane masahiro sadarmoto shoozo yoshimura s...
1          anon  “chapter  frequencydomain behavior elect...
2          huang ximing chen jingchao fang xiaotong yan c...
3          demars b l harper pitt ja slaughter r  “impact...
4          ling jian ming zhu li guo yuan jie chai zhen l...
                                 ...                        
1999995    allred h  “histochemical observations lipids s...
1999996    pavlidis e  “current opinion laparoscopic repa...
1999997     “book reviews” american journal cancer  ameri...
1999998    afarinkia k v vinader  “halogenation” sixmembe...
1999999    boonnak n c karalai chantrapromma c ponglimano...
Name: citationStringAnnotated, Length: 2000000, dtype: object

In [12]:
def build_lsa(dataset, dim, queue ):
        tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2')
        svd = TruncatedSVD(n_components=dim)
    
        transformed_x_train = tfidf_vec.fit_transform(dataset)
    
        print('TF-IDF output shape:', transformed_x_train.shape)
    
        data_svd = svd.fit_transform(transformed_x_train)
    
        print('LSA output shape:', data_svd.shape)
    
        explained_variance = svd.explained_variance_ratio_.sum()
        print("Sum of explained variance ratio: %d%%" % (int(explained_variance * 100)))

        queue.put((tfidf_vec, transformed_x_train,svd, data_svd))
         

In [14]:
queue1 = queue.Queue()
# queue2 = queue.Queue()
thread_lsa1 = threading.Thread(target = build_lsa, args = (result,100, queue1,)) 
# thread_lsa2 = threading.Thread(target = build_lsa, args = (df2['Preprocess'],100, queue2, )) 
thread_lsa1.start()
# thread_lsa2.start()
thread_lsa1.join()
# thread_lsa2.join()

TF-IDF output shape: (2000000, 67403)
LSA output shape: (2000000, 100)
Sum of explained variance ratio: 11%


In [15]:
tfidf_vec1, transformed_x_train1,svd1, data_svd1 = queue1.get()
# tfidf_vec2, transformed_x_train2,svd2, data_svd2 = queue2.get()

In [18]:
l=data_svd1[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  12.50495918385431
Topic  1  :  -2.3801169351247506
Topic  2  :  0.9087670276448259
Topic  3  :  2.3532442809181338
Topic  4  :  1.3929844409035361
Topic  5  :  7.715277200794582
Topic  6  :  11.032976835980863
Topic  7  :  -2.701808012213281
Topic  8  :  -4.530366863596789
Topic  9  :  4.5324967114711745
Topic  10  :  0.9873632071705927
Topic  11  :  -1.1687506198094813
Topic  12  :  -3.503751116769365
Topic  13  :  0.782043290120966
Topic  14  :  -2.221997098628227
Topic  15  :  4.103362993443693
Topic  16  :  2.0952742142311056
Topic  17  :  2.0209573869858737
Topic  18  :  -3.309032441126643
Topic  19  :  -3.239683317850898
Topic  20  :  -3.34447398541732
Topic  21  :  2.5591770600146604
Topic  22  :  -0.7106381115371223
Topic  23  :  -0.5476148474544853
Topic  24  :  -3.0592715006473816
Topic  25  :  -4.762068040783994
Topic  26  :  5.570898342150846
Topic  27  :  1.8215739337839876
Topic  28  :  2.7883089455119694
Topic  29  :  -2.421747630000627
Topic  3

In [135]:
vocab = tfidf_vec1.get_feature_names()
a = []
d = []
for i, comp in enumerate(svd1.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    a.insert(i,np.array(sorted_words)[:,1])
    d.append(" ".join(np.array(sorted_words)[:,0]))
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
available journal pp online doi international ieee springer american et 

Topic 1: 
ieee conference international th systems information electrical proceedings symposium transactions 

Topic 2: 
springer heidelberg berlin pp und der nature ieee netherlands geotechnical 

Topic 3: 
de et la en gruyter al du revista des el 

Topic 4: 
press university oxford available online anon oup pp ieee internet 

Topic 5: 
available online anon internet lancet elsevier al et bv news 

Topic 6: 
et al press university oxford elsevier bv zhang clinical patients 

Topic 7: 
elsevier bv pp vol lancet journal editorial board anon index 

Topic 8: 
doi lancet notes news review elsevier internet bv na queries 

Topic 9: 
pp vol american lt review lancet disponible en news ligne 

Topic 10: 
science technology notes elsevier queries food engineering review bv internet 

Topic 11: 
lancet notes elsevier bv news queries journal ieee anon berlin 

Topic 12: 
review international new journal book fac

In [138]:
b = pd.DataFrame(a)
b

,0,1,2,3,4,5,6,7,8,9
0,0.32882199412308266,0.31895110367102747,0.2787109982689526,0.1998770102520009,0.19511859297043607,0.17484375453276066,0.16376039169937814,0.1539404401280229,0.1426867513303784,0.14117164522643044
1,0.607822173626732,0.35585934968415467,0.30147434342808516,0.15646899461300237,0.10872845912817489,0.10157624567344352,0.08648944068045901,0.0863102460079349,0.08274913722537879,0.08179650733812671
2,0.5036118626571917,0.38764477617227594,0.38348148727184916,0.2492515038478758,0.113627487601286,0.0819567197556083,0.08081998580616627,0.061496854551808255,0.05999142091267861,0.05791228444059227
3,0.7920405633395564,0.20207823499100647,0.16897949990056574,0.15857778061871347,0.13701416373746123,0.13230510898177503,0.07941339158358193,0.07526168176132025,0.07494103974282094,0.061217287376670586
4,0.41815603320213895,0.3837689762154347,0.2707024419925105,0.251917997514554,0.20490968446433336,0.16404972503679136,0.11150082795905783,0.10822328419648719,0.10087966015911363,0.09113368884676899
...,...,...,...,...,...,...,...,...,...,...
95,0.1305373281876518,0.12349457638802887,0.11182936272842563,0.10431339116132228,0.09749968401502059,0.09713641532928526,0.09162760473747532,0.0885173406223587,0.08604284324648545,0.08485958907963964
96,0.13695084200279042,0.12190582661261935,0.11957653385312693,0.11247937439835004,0.09329635421524776,0.08779707241161912,0.08133729890544683,0.08068441280411491,0.07949362938513718,0.07549125947615025
97,0.1472512547965891,0.11503891880900877,0.10071600715284905,0.09948472952030338,0.09724751823656973,0.09658333192172182,0.09511923503230785,0.08780113289831899,0.08475144440445898,0.084558710014494
98,0.12129088217323261,0.11309604729912513,0.10697272338366141,0.09787895583386394,0.09634599410076343,0.09555002987831959,0.08875999699239744,0.08518859669035168,0.08257999508919785,0.08163197570732074


In [139]:
list_app_name = list(d)

dic_recommended_1 = {}

for index in range(b.shape[0]):

    similarities_1 = linear_kernel(a[index].reshape(1, -1) ,a).flatten()
    related_docs_indices_1 = (-similarities_1).argsort()[:10]

    dic_recommended_1.update({list_app_name[index]:[list_app_name[i] for i in related_docs_indices_1]})

df_content_based_results_1 = pd.DataFrame(dic_recommended_1)
df_content_based_results_1.reset_index(inplace=True)
df_content_based_results_1 = df_content_based_results_1.T
df_content_based_results_1

,0,1,2,3,4,5,6,7,8,9
index,0,1,2,3,4,5,6,7,8,9
available journal pp online doi international ieee springer american et,springer heidelberg berlin pp und der nature i...,ieee conference international th systems infor...,press university oxford available online anon ...,doi lancet notes news review elsevier internet...,de et la en gruyter al du revista des el,available journal pp online doi international ...,lt em disponível wiley john na review sons rea...,science technology notes elsevier queries food...,bmj cheminform abstract nono clinical medical ...,lt em disponível lancet research health inform...
ieee conference international th systems information electrical proceedings symposium transactions,de et la en gruyter al du revista des el,ieee conference international th systems infor...,springer heidelberg berlin pp und der nature i...,doi lancet notes news review elsevier internet...,available online anon internet lancet elsevier...,science technology notes elsevier queries food...,et al press university oxford elsevier bv zhan...,press university oxford available online anon ...,bmj cheminform abstract nono clinical medical ...,lt em disponível lancet research health inform...
springer heidelberg berlin pp und der nature ieee netherlands geotechnical,springer heidelberg berlin pp und der nature i...,ieee conference international th systems infor...,de et la en gruyter al du revista des el,available online anon internet lancet elsevier...,doi lancet notes news review elsevier internet...,press university oxford available online anon ...,et al press university oxford elsevier bv zhan...,science technology notes elsevier queries food...,lt em disponível lancet research health inform...,bmj cheminform abstract nono clinical medical ...
de et la en gruyter al du revista des el,de et la en gruyter al du revista des el,doi lancet notes news review elsevier internet...,ieee conference international th systems infor...,science technology notes elsevier queries food...,springer heidelberg berlin pp und der nature i...,available online anon internet lancet elsevier...,et al press university oxford elsevier bv zhan...,press university oxford available online anon ...,nd dois crossref inactive health cancer encycl...,bmj cheminform abstract nono clinical medical ...
...,...,...,...,...,...,...,...,...,...,...
the communications philosophy australian vol nursing national transfer data economic,press university oxford available online anon ...,springer heidelberg berlin pp und der nature i...,lt em disponível wiley john na review sons rea...,ieee conference international th systems infor...,available journal pp online doi international ...,doi lancet notes news review elsevier internet...,de et la en gruyter al du revista des el,bmj cheminform abstract nono clinical medical ...,science technology notes elsevier queries food...,lt em disponível lancet research health inform...
design association page quarterly historical theory biology history communication public,press university oxford available online anon ...,springer heidelberg berlin pp und der nature i...,ieee conference international th systems infor...,doi lancet notes news review elsevier internet...,lt em disponível wiley john na review sons rea...,available journal pp online doi international ...,de et la en gruyter al du revista des el,science technology notes elsevier queries food...,lt em disponível lancet research health inform...,bmj cheminform abstract nono clinical medical ...
british water index networks chemical mass encyclopedia computer development book,press university oxford available online anon ...,doi lancet notes news review elsevier internet...,ieee conference international th systems infor...,de et la en gruyter al du revista des el,springer heidelberg berlin pp und der nature i...,lt em disponível wiley john na review sons rea...,available journal pp online doi international ...,science technology notes elsevier queries food...,bmj cheminform abstract nono 